**The purpose of this notebook is to create a DATASET that includes**

** Characteristics of patients like - **

*age

*sex

*Country

and so




**The condition of the patients and their characteristics - **

* Disease time (from diagnosis date)

* Have been cured
 
* Deaths


The database is designed to allow easy exploration of the data

Anyone interested can use and donate

In [ ]:
import pandas as pd 
import  numpy as np
from ds_exam import *

In [ ]:
france = pd.read_csv("/kaggle/input/coronavirus-france-dataset/patient.csv")
tunisia = pd.read_csv("../input/coronavirus-tunisia/Coronavirus_Tunisia.csv")
japan = pd.read_csv("/kaggle/input/close-contact-status-of-corona-in-japan/COVID-19_Japan_Mar_07th_2020.csv")
indonesia = pd.read_csv("/kaggle/input/indonesia-coronavirus-cases/patient.csv")
korea = pd.read_csv("/kaggle/input/coronavirusdataset/PatientInfo.csv")
Hubei = pd.read_csv("/kaggle/input/covid19official/Hubei.csv")
outside_Hubei = pd.read_csv("/kaggle/input/covid19official/outside_Hubei.csv")

In [ ]:
datasets = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
datasets_name = ["france", "tunisia", "japan", "indonesia", "korea", "Hubei", "outside_Hubei"]

garbge = [print("\n"+datasets_name[i], [i for i in datasets[i].columns]) for i in range(len(datasets_name))]

change name of col

In [ ]:
france.rename(columns={"health":"severity_illness","status":"treatment","infection_reason":"infection_place"}
              , inplace = True)

tunisia.rename(columns={"date":"confirmed_date", "gender":"sex", "situation":"severity_illness", 
                        "return_from":"infection_place", "health":"background_diseases"}, inplace = True)

japan.rename(columns={"No.":"id", "Fixed date":"confirmed_date","Age":"age", "residence":"region",
                      "Surrounding patients *":"infected_by"}, inplace = True)

indonesia.rename(columns={"patient_id":"id","gender": "sex", "province":"region", "hospital":"hospital_name",
                          "contacted_with":"infected_by", "current_state":"status"}, inplace = True)

korea.rename(columns={"patient_id":"id", "disease":"background_diseases_binary", "state":"severity_illness",
                      "province":"region", "infection_case" :"infection_place",
                      "symptom_onset_date":"date_onset_symptoms"}, inplace = True)

Hubei = Hubei.rename(columns={ "province":"region","date_confirmation": "confirmed_date",
                              "chronic_disease_binary":"background_diseases_binary", 
                              "chronic_disease":"background_diseases", "outcome":"severity_illness"})

outside_Hubei = outside_Hubei.rename(columns={ "province":"region", "date_confirmation": "confirmed_date",
                              "chronic_disease_binary":"background_diseases_binary", 
                              "chronic_disease":"background_diseases", "outcome":"severity_illness" })


# datasets.shape

In [ ]:
o = []
for i in range(len(datasets)):
    print(datasets_name[i],datasets[i].shape)
    o.append(datasets[i].shape[0])
print("\nnum of i " + str(sum(o)))

# exam_df = columns vs dfs

In [ ]:
datasets = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
columns_name = Exam.build_columns_name_ls(datasets)
exam_df = Exam.df_exam_columns_dfs(datasets,datasets_name,columns_name)

In [ ]:
def full_common(exam_df):
    """
    Returns columns that all DATASETS have
    """
    full_common = []
    for j in exam_df.columns:
        boolyan = exam_df[j].all()
        if boolyan == True:
            full_common.append(j)
    return full_common

In [ ]:
common = []
unique = []
blank = []
for i in exam_df.columns:
    if exam_df[i].value_counts()[True]>1:
        common.append(i)
    elif exam_df[i].value_counts()[True]==1:
        unique.append(i)
    else:
        blank.append(i)
        
        
print(common)
print(unique)
print(blank)   

# drop feature

Tests for columns' usefulness before drop

country_new

In [ ]:
for x in [outside_Hubei, Hubei]:
    l = x.index[x.country_new.notnull() == True]
    p = []
    for i in l:
        if x.country_new[i] == x.country[i]:
            p.append(i)

    print("country_new == country",len(p))
    print("country_new.notnull",len(l),"\n")

The column has no new information to give

ID

In [ ]:
for x in [outside_Hubei, Hubei]:
    m =[]
    for i in range(len(x)):
        if x.ID[i] != str(i+1):
            m.append(i)
    print(m[0],x.ID[m[0]])

As you can see the ID is not arranged in any numerical order.
and because there is no column that needs another row identifier
I drop ID

# drop

In [ ]:
france = france.drop(["departement","source","comments","contact_number"],axis=1)


indonesia = indonesia.drop(["origin"],axis=1)

korea = korea.drop(["age","contact_number"],axis=1)

Hubei = Hubei.drop(["ID",'location', 'admin3', 'admin2', "admin1" ,'latitude', 'longitude',
                    'geo_resolution','admin_id', "country_new","source","additional_information","geo_resolution"
                    ,"notes_for_discussion"],axis=1)

outside_Hubei = outside_Hubei.drop(["ID",'location', 'admin3', 'admin2', "admin1" ,'latitude', 'longitude',
                                    'geo_resolution', 'admin_id', "country_new", "data_moderator_initials",
                                    "source","additional_information","geo_resolution",
                                    "notes_for_discussion"],axis=1)

# Examining values - v1 

In [ ]:
def examining_values_by_col (dfs, dfs_name, col):
    """
    Prints values of each DF per column
    """
    counter = 0
    
    for i in datasets:
        if col in i.columns:
            print("\n" + dfs_name[counter])
            print(i[col].value_counts())
        counter =counter + 1

In [ ]:
datasets = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
columns_name = Exam.build_columns_name_ls(datasets)
exam_df = Exam.df_exam_columns_dfs(datasets,datasets_name,columns_name)

for j in exam_df.columns[1:len(exam_df.columns)]:
    print(j)
    examining_values_by_col (datasets , datasets_name , j) 

# format col

>datetime

In [ ]:
## TODO make FUN to_date

cols = ["confirmed_date","released_date", "deceased_date"]

france[cols] = france[cols].apply(pd.to_datetime)
indonesia[cols] = france[cols].apply(pd.to_datetime)
japan[cols] = france[cols].apply(pd.to_datetime)
korea[cols] = korea[cols].apply(pd.to_datetime)

# different
tunisia_col = ["confirmed_date"]
#  tunisia_col  = "return_date" problame
korea_col = ["date_onset_symptoms"] + cols

tunisia[tunisia_col] = tunisia[tunisia_col].apply(pd.to_datetime)
korea[korea_col] = korea[korea_col].apply(pd.to_datetime)

# לא עובד . בעיה אם השעות אם אני יעשה את זה - יותר נוח מאחרי בניית התכונות
Hubei_col = ["confirmed_date", "date_death_or_discharge", "date_onset_symptoms"]
Hubei[Hubei_col] = Hubei[Hubei_col].apply(pd.to_datetime)

# "confirmed_date", "date_onset_symptoms"
Hubei_col = ["date_death_or_discharge"]
outside_Hubei[Hubei_col] = outside_Hubei[Hubei_col].apply(pd.to_datetime)

In [ ]:
examining_values_by_col(datasets, datasets_name, "travel_history_dates")

In [ ]:
def make_ls_of_str_datatime(ls):
        for i in range(len(ls)):
            ls[i] = pd.to_datetime(ls[i], errors='ignore').date()
        return ls
        
def  time_range_extremity(ls, earliest=False):
    df = pd.DataFrame({"series":ls})
    if earliest == True:
        value = df.series.max()
    else:
        value = df.series.min()
    return value

In [ ]:
outside_Hubei.travel_history_dates[7957]

In [ ]:
for indx in outside_Hubei.index[outside_Hubei.travel_history_dates.notnull()]:
    print(indx)

In [ ]:
for indx in outside_Hubei.index[outside_Hubei.travel_history_dates.notnull()]:
    i = outside_Hubei.travel_history_dates[indx] 
    i_split_1 = i.split(',')
    
    if len(i_split_1) > 1:
        i_split_1 = make_ls_of_str_datatime(i_split_1)
        value = time_range_extremity(i_split_1)
        outside_Hubei.loc[indx, 'travel_history_dates'] = value
    
    i_split_2 = i_split_1[0].split('-')
    if len(p) >1:
        for i in range(len(p)):
            p[i] = pd.to_datetime(p[i], errors='ignore').date()
        d = pd.DataFrame({"selfs":p})
        print(p,indx)
        print(d.selfs.max())
        print(p[0],p[1])
        print("------")
    else:
        p[0] = pd.to_datetime(p[0], errors='ignore').date()


In [ ]:
date_string = '2019-04-17'
time_stamp = pd.to_datetime(date_string)
print(time_stamp)

In [ ]:
examining_values_by_col(datasets, datasets_name, "travel_history_dates")

sex

In [ ]:
tunisia_sex = {"F":"female", "M":"male",np.nan:np.nan}
tunisia.sex = [tunisia_sex[item] for item in  tunisia.sex] 

japan_sex = {"Woman":"female", "Man":"male",np.nan:np.nan, "Checking":np.nan, "investigating":np.nan}
japan.sex = [japan_sex[item] for item in  japan.sex] 

france_sex = {"female":"female", "male":"male","Female":"female", "Male":"male", "male\xa0?":"male", 
              np.nan:np.nan }
france.sex = [france_sex[item] for item in  france.sex] 


In [ ]:
examining_values_by_col(datasets, datasets_name, "sex")

In [ ]:
def update_index(df, col, index, data):
    """
    Value change according index
    
    df: df
    col : col you want to change
    index: pd.index
    data: data you want to into
    
    """
    for i in index:
        df[col][i]=data

background_diseases_binary

In [ ]:
# indexs = korea.index[korea.background_diseases_binary == True]
# p = [i for i in indexs]

In [ ]:
examining_values_by_col (datasets, datasets_name, "background_diseases_binary")

In [ ]:
tunisia["background_diseases_binary"] = np.nan
indexs = tunisia.index[tunisia.background_diseases.notnull()]
update_index(tunisia,"background_diseases_binary",indexs,1) 

background_diseases

In [ ]:
words =[]
for i in outside_Hubei.background_diseases[outside_Hubei.background_diseases.notnull()]:
    [words.append(i) for i in i.split(" ")]

words = list(dict.fromkeys(words))
print(words)

In [ ]:
# import nltk
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords

In [ ]:
words =[]
for i in outside_Hubei.severity_illness[outside_Hubei.severity_illness.notnull()]:
    [words.append(i) for i in i.split(" ")]

words = list(dict.fromkeys(words))
print(words)

#                                                     Complete features

severity_illness

In [ ]:
for x in [indonesia,japan]:   
    x["severity_illness"] = np.nan

    indexs = x.index[x.deceased_date.notnull()]
    update_index(x,"severity_illness",indexs,"deceased") 

    indexs = x.index[x.released_date.notnull()]
    update_index(x,"severity_illness",indexs,"cured") 



severity_illness_tunisia = {"Critical":"critical", "In progress":"good", "Stable":"good", "Cured":"cured" }
tunisia.Severity_illness = [severity_illness_tunisia[item] for item in  tunisia.severity_illness]


indexs = korea.index[korea.severity_illness.isnull()] 
update_index(korea,"severity_illness",indexs,np.nan)
severity_illness_korea = {"deceased":"deceased", "isolated": np.nan, "released":"cured", np.nan : np.nan }
korea.severity_illness = [severity_illness_korea[item] for item in  korea.severity_illness]


In [ ]:
datasets2 = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
examining_values_by_col(datasets2, datasets_name, "severity_illness")

released_date

In [ ]:
for x in [outside_Hubei, Hubei]:
    x["released_date"] = np.nan 
    indexs_released = x.index[x.severity_illness =="discharged"]

    for i in indexs_released:
        x["released_date"][i] = x["date_death_or_discharge"][i]

deceased_date

In [ ]:
for x in [outside_Hubei, Hubei]:
    x["deceased_date"] = np.nan 
    indexs_released = x.index[x.severity_illness =="died"]

    for i in indexs_released:
        x["deceased_date"][i] = x["date_death_or_discharge"][i]

format date

In [ ]:
for x in [outside_Hubei, Hubei]:
    type(Hubei.deceased_date[2])
    col = ["released_date","deceased_date"]
    x[col] = x[col].apply(pd.to_datetime)

test

In [ ]:
for x in [outside_Hubei, Hubei]:
    l = x.date_death_or_discharge.notnull().sum()
    y = x.severity_illness.notnull().sum()
    p = x.released_date.notnull().sum() +x.deceased_date.notnull().sum()
    print(l,y,p)

In [ ]:
for x in [outside_Hubei]:
    complete_features =list(x.index[x.released_date.notnull()]) + list(x.index[x.deceased_date.notnull()])
    date_death_or_discharge = list(x.index[x.date_death_or_discharge.notnull()])
    severity_illness = list(x.index[x.severity_illness.notnull()])

    if complete_features == date_death_or_discharge:
        print("==")
    else:
        print("not ==")
    
    for i in severity_illness:
        if i not in complete_features:
            print(i)

age

exam 

In [ ]:
# outside_Hubei_age = {"investigating":np.nan, "Checking":np.nan, "Under 10":"0s", "Under teens":"0s","305":"30s",
#             "10s":"10s","20s":"20s", "30-39":"30s", "40-49":"40s", "50-59":"50s", "60-69":"60s", "70s":"70s" ,
#              "80s":"80s","90s":"90s" }
# outside_Hubei.age = [outside_Hubei_age[item] for item in outside_Hubei.age] 

In [ ]:
Hubei.age.value_counts()

In [ ]:
# Hubei_age = {"15-88":np.nan, "25-89":np.nan, "21-39":np.nan, "40-49":"40s", "50-59":"50s", "60-69":"60s",
#              "70-82":"70s" }
# Hubei.age = [Hubei_age[item] for item in Hubei.age] 

In [ ]:
japan.age.value_counts()

In [ ]:
japan_age = {"investigating":np.nan, "Checking":np.nan, "Under 10":"0s", "Under teens":"0s","305":"30s",
            "10s":"10s","20s":"20s", "30s":"30s", "40s":"40s", "50s":"50s", "60s":"60s", "70s":"70s" ,
             "80s":"80s","90s":"90s" }
japan.age = [japan_age[item] for item in japan.age] 

In [ ]:
def birth_year_to_age(data):
    age_ls = []

    for i in range(len(data)):
        age_ls.append(data.confirmed_date[i].year - data.birth_year[i])
    return age_ls

korea["age"] = birth_year_to_age(korea)
france["age"] = birth_year_to_age(france)

country

In [ ]:
tunisia["country"] = ["tunisia" for i in range(len(tunisia))]
japan["country"] = ["japan" for i in range(len(japan))]
indonesia["country"] = ["indonesia" for i in range(len(indonesia))]

outside_Hubei data VS country data

In [ ]:
print(len(korea))
print(outside_Hubei.country.value_counts()["South Korea"])
print()

print(len(france))
print(outside_Hubei.country.value_counts()["France"])

status

In [ ]:
france["status"] =[np.nan for i in np.arange(len(france))]
france.status = france.status.astype("object")
france["status"].dtype

In [ ]:
deceased = france.index[france.deceased_date.notnull() == True]
released = france.index[france.released_date.notnull() == True]
isolated = france.index[(france.released_date.notnull() == False)&(france.deceased_date.notnull() == False)]

In [ ]:
for i in deceased:
    france.at[ i, 'status'] = "deceased"

for i in released:
    france.at[ i, 'status'] = "released" 
    
for i in isolated:
    france.at[ i, 'status'] = "isolated"

In [ ]:
france.status.isnull().sum()

infection_place

In [ ]:
print(Hubei["wuhan(0)_not_wuhan(1)"].value_counts())
print(outside_Hubei["travel_history_location"].value_counts())

infection_case

= Community \abroad \ Nan

In [ ]:
# TODO

#  Garbage drop 
- Features that have only one dataset or  built with Engineered another feature with them

In [ ]:
france = france.drop(["birth_year", "treatment","group"],axis=1)
tunisia = tunisia.drop(["hospital_place"],axis=1)
japan = japan.drop(["Close contact situation"],axis=1)
korea = korea.drop(["birth_year","global_num"],axis=1)
Hubei = Hubei.drop(["date_death_or_discharge"],axis=1)

# drop Non-baked features

In [ ]:
france = france.drop(["infection_place", "infected_by","infection_order"],axis=1)
tunisia = tunisia.drop(["hospital_name"],axis=1)
indonesia = indonesia.drop(["infected_by","hospital_name"],axis=1)
korea = korea.drop(["infection_place", "infected_by","infection_order"],axis=1)

**> Feature sum**

In [ ]:
datasets2 = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
columns_name = Exam.build_columns_name_ls(datasets2)
exam_df = Exam.df_exam_columns_dfs(datasets2, datasets_name, columns_name)
print(columns_name)
exam_df.infection_place

unipue

common feature

In [ ]:
datasets3 = [france, tunisia, japan, indonesia, korea, Hubei, outside_Hubei]
columns_name = Exam.build_columns_name_ls(datasets3)
exam_df2 = Exam.df_exam_columns_dfs(datasets3,datasets_name,columns_name)

In [ ]:
for i in exam_df2.columns:
    print("\n"+i)
    examining_values_by_col (datasets, datasets_name, i)

# build final DS

In [ ]:
exam_df.sex

In [ ]:
for i in dfs:
    print(i[col].isnull().sum())

In [ ]:
for i in datasets:
    print(i.sex.isnull().sum())

In [ ]:
datasets_final = [france, tunisia, japan, indonesia, korea]
final_DS = pd.concat(datasets_final, axis=0)

In [ ]:
final_DS.status.value_counts()

# orgnaze DS

index

In [ ]:
final_DS.index = range(len(final_DS))

In [ ]:
final_DS.to_csv(r'/kaggle/working/Characteristics_Corona_patients1.csv', index = False)

In [ ]:
final_DS.to_csv()

infected by